# Практическая работа №5

Выполнил студент группы М06-106н Кочетков Дмитрий Андреевич.

## Постановка задачи

Необходимо выполнить все следующие задачи:

1. Сконструировать класс `BinaryHeap`, реализующий следующие операции над двоичной max-кучей:
- `push` - добавление элемента в кучу;
- `pop` – получить значение и удалить корневой элемент из кучи.
- (дополнительное задание – по желанию) Реализовать конструктор класса, которому в качестве параметра передается список элементов, из которых нужно построить кучу.

2. Реализовать класс `TaskQueue` для работы со списком задач. Для каждой задачи задается название и приоритет (число от 1 до 4: 1 – низкий приоритет, 2 – средний приоритет, 3 – высокий приоритет, 4 – критическая задача). Класс должен обеспечивать возможность добавления задач в очередь, извлечение задачи из очереди, смену приоритета задачи из очереди. Для реализации использовать двоичную кучу. Для работы с двоичной кучей использовать библиотеку `heapq`.

3. Сравнить время работы очереди с приоритетом на основе 1) отсортированного списка (list) и 2) двоичной кучи (библиотека `heapq`). Для этого рассчитать время заполнения и извлечения всех данных из очереди с приоритетом с использованием обоих методов (списка и кучи) на списках разной длины. Построить на одном графике зависимости времени работы с очередью от размера списка для обоих реализаций. Сделать вывод.


## Задача 1

### Реализация BinaryHeap

### Тестирование BinaryHeap

## Задача 2

In [1]:
from heapq import heappush, heappop, heapify

class Task:
    def __init__(self, name: str, priority: int):
        if not isinstance(priority, int):
            raise ValueError

        if priority < 1 or priority > 4:
            raise ValueError
        
        self.id = None
        self.name = name
        self.priority = priority
    
    def __lt__(self, other):
        return self.priority < other.priority

    def __str__(self):
        return '[ID: {}, priority: {}] {}'.format(self.id, self.priority, self.name)

    def __repr__(self):
        return '[ID: {}, priority: {}] {}'.format(self.id, self.priority, self.name)

class TaskQueue:
    def __init__(self):
        self.tasks = []
        self.next_id = 1

    def insert(self, task: Task):
        heappush(self.tasks, task)
        task.id = self.next_id
        self.next_id += 1

    def pop(self):
        return heappop(self.tasks)
    
    def changePriority(self, id: int, new_priority: int):
        selected_task = None
        for task in self.tasks:
            if task.id == id:
                selected_task = task

        selected_task.priority = new_priority
        heapify(self.tasks)

    def __str__(self):
        result = ''
        for item in sorted(self.tasks):
            result += str(item) + '\n'
        return result

    def __repr__(self):
        return self.__str__()

In [2]:
# Названия тасков не смог придумать, поэтому взял из GitHub-репозитория Angular
queue = TaskQueue()
queue.insert(Task('control.hasError() return unexpected value', 3))
queue.insert(Task('Expose flushPeriodic from FakeAsync module', 4))
queue.insert(Task('extract-i18n JSON format add support for meaning and description', 2))
queue.insert(Task('Input-Placeholder is empty after resetting a auto-completed input-field in firefox', 4))
queue.insert(Task('ProvideIn singleton service among multiple angular instances', 1))
queue.insert(Task('Animation not working correctly with Angular routing and component reusing (caching)', 3))
queue.insert(Task('Angular.io website is not GDPR compliant', 1))

queue

[ID: 5, priority: 1] ProvideIn singleton service among multiple angular instances
[ID: 7, priority: 1] Angular.io website is not GDPR compliant
[ID: 3, priority: 2] extract-i18n JSON format add support for meaning and description
[ID: 6, priority: 3] Animation not working correctly with Angular routing and component reusing (caching)
[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 4, priority: 4] Input-Placeholder is empty after resetting a auto-completed input-field in firefox
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module

In [3]:
queue.pop()
queue

[ID: 7, priority: 1] Angular.io website is not GDPR compliant
[ID: 3, priority: 2] extract-i18n JSON format add support for meaning and description
[ID: 6, priority: 3] Animation not working correctly with Angular routing and component reusing (caching)
[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 4, priority: 4] Input-Placeholder is empty after resetting a auto-completed input-field in firefox
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module

In [4]:
queue.pop()
queue

[ID: 3, priority: 2] extract-i18n JSON format add support for meaning and description
[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 6, priority: 3] Animation not working correctly with Angular routing and component reusing (caching)
[ID: 4, priority: 4] Input-Placeholder is empty after resetting a auto-completed input-field in firefox
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module

In [5]:
queue.pop()
queue

[ID: 6, priority: 3] Animation not working correctly with Angular routing and component reusing (caching)
[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module
[ID: 4, priority: 4] Input-Placeholder is empty after resetting a auto-completed input-field in firefox

In [6]:
queue.pop()
queue

[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 4, priority: 4] Input-Placeholder is empty after resetting a auto-completed input-field in firefox
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module

In [7]:
queue.changePriority(4, 1)
queue

[ID: 4, priority: 1] Input-Placeholder is empty after resetting a auto-completed input-field in firefox
[ID: 1, priority: 3] control.hasError() return unexpected value
[ID: 2, priority: 4] Expose flushPeriodic from FakeAsync module

## Вывод

В ходе практической работы была реализована двоичная куча. Проведено сравнение реализованной кучи с библиотечной реализацией `heapq`.